In [1]:
import numpy as np
import modern_robotics as mr

def FeedbackControl(X, X_d, X_d_next, K_p, K_i, delt, joint_angles):
    l = 0.47/2
    w = 0.3/2
    r = 0.0475
    error = np.zeros((6,1))
    
    phi = joint_angles[0,0]
    x = joint_angles[0,1]
    y = joint_angles[0,2]
    
    F = r/4 * np.asarray([[-1/(l+w), 1/(l+w), 1/(l+w), -1/(l+w)], [1, 1, 1, 1], [-1, 1, -1, 1]])
    
    F6 = np.zeros((6,4))
    F6[2:5, :] = F
    
    M = np.asarray([[1, 0, 0, 0.033], [0, 1, 0, 0], [0, 0, 1, 0.6546], [0, 0, 0, 1]])

    T_b0 = np.asarray([[1, 0, 0, 0.1662], [0, 1, 0, 0], [0, 0, 1, 0.0026], [0, 0, 0, 1]])
    
    T_sb = np.asarray([[np.cos(phi), -np.sin(phi), 0, x], [np.sin(phi), np.cos(phi), 0, y], [0, 0, 1, 0.0963], [0, 0, 0, 1]])
    
    
    Vd = mr.se3ToVec(1/delt * mr.MatrixLog6(mr.TransInv(X_d) @ X_d_next)).reshape((6,1))
    Xerr = mr.se3ToVec(mr.MatrixLog6(mr.TransInv(X) @ X_d)).reshape((6,1))
    ad = mr.Adjoint(mr.TransInv(X) @ X_d)
    error += Xerr*delt
    
    Blist = np.asarray([[0, 0, 1, 0, 0.033, 0], [0, -1, 0, -0.5076, 0, 0], 
                        [0, -1, 0, -0.3526, 0, 0], [0, -1, 0, -0.2176, 0, 0], 
                       [0, 0, 1, 0, 0, 0]]).T
    
    Jarm = mr.JacobianBody(Blist, joint_angles[0,3:])
    
    T_0e = mr.FKinBody(M, Blist, joint_angles[0,3:])
    
    ad2 = mr.Adjoint(mr.TransInv(T_0e)@mr.TransInv(T_b0))
    
    Jbase = ad2 @ F6
    
    J = np.concatenate((Jbase,Jarm), axis=1)
    
    V = ad @ Vd + K_p @ Xerr + K_i @ error
    
    output = np.linalg.pinv(J) @ V
    
    print("Vd: ")
    print(Vd)
    print("ad@Vd: ")
    print(ad@Vd)
    print("V: ")
    print(V)
    print("Xerr: ")
    print(Xerr)
    print("Jacobian: ")
    print(J)
    print("Outputs: ")
    print(output)
    print("Jbase: ")
    print(Jbase)
    print("Jarm: ")
    print(Jarm)
    
    
    
X = np.asarray([[0.17, 0, 0.985, 0.387], [0, 1, 0, 0], [-0.985, 0, 0.17, 0.57], [0, 0, 0, 1]])
X_d = np.asarray([[0, 0, 1, 0.5], [0, 1, 0, 0], [-1, 0, 0, 0.5], [0, 0, 0, 1]])
X_d_next = np.asarray([[0, 0, 1, 0.6], [0, 1, 0, 0], [-1, 0, 0, 0.3], [0, 0, 0, 1]])
K_p = np.eye(6)
K_i = np.zeros((6,6))
delt = 0.01
joint_angles = np.asarray([[0,0,0,0,0,0.2,-1.6,0]])
FeedbackControl(X, X_d, X_d_next, K_p, K_i, delt, joint_angles)
    

Vd: 
[[ 0.]
 [ 0.]
 [ 0.]
 [20.]
 [ 0.]
 [10.]]
ad@Vd: 
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [21.4 ]
 [ 0.  ]
 [ 6.45]]
V: 
[[ 0.        ]
 [ 0.17085513]
 [ 0.        ]
 [21.47945351]
 [ 0.        ]
 [ 6.55669436]]
Xerr: 
[[0.        ]
 [0.17085513]
 [0.        ]
 [0.07945351]
 [0.        ]
 [0.10669436]]
Jacobian: 
[[ 3.03953650e-02 -3.03953650e-02 -3.03953650e-02  3.03953650e-02
  -9.85449730e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
   0.00000000e+00]
 [-5.24249304e-03  5.24249304e-03  5.24249304e-03 -5.24249304e-03
   1.69967143e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00]
 [ 2.01835982e-03  2.01835982e-03  2.01835982e-03  2.01835982e-03
   0.00000000e+00 -2.40002972e-01 -2.13658064e-01 -2.17600000e-01
   0.00000000e+00]
 [-2.38059359e-02  2.38059359e-02  5.59359256e-05 -5.59359256e-05
   2.20613502e-01  0.